# Семинар 2. Классификация текстов

## Анализ тональности отзывов

### Считаем и изучим датасет

Скачайте и разархивируйте датасет в папку "sentiment labelled sentences" рядом с вашим jupyter-ноутбуком<br/>
https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences

In [538]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 500)

In [539]:
!ls

02. Классификация текстов - идеи.ipynb
02. Классификация текстов.ipynb
__pycache__
sentiment labelled sentences
twitter_settings.py


In [541]:
!ls "sentiment labelled sentences"

amazon_cells_labelled.txt readme.txt
imdb_labelled.txt         yelp_labelled.txt


In [695]:
ds = pd.read_csv("sentiment labelled sentences/amazon_cells_labelled.txt", 
                 delimiter="\t", names=("text", "is_positive"))

In [696]:
ds.head()

,text,is_positive
0,So there is no way for me to plug it in here in the US unless I go by a converter.,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!,0
4,The mic is great.,1


In [697]:
ds.is_positive.value_counts()

1    500
0    500
Name: is_positive, dtype: int64

### Построим Bag of words по текстам

In [698]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()#ngram_range=(1,2))
X = vectorizer.fit_transform(ds.text)

In [699]:
# какие получились признаки
vectorizer.get_feature_names()

['10',
 '100',
 '11',
 '12',
 '13',
 '15',
 '15g',
 '18',
 '20',
 '2000',
 '2005',
 '2160',
 '24',
 '2mp',
 '325',
 '350',
 '375',
 '3o',
 '42',
 '44',
 '45',
 '4s',
 '50',
 '5020',
 '510',
 '5320',
 '680',
 '700w',
 '8125',
 '8525',
 '8530',
 'abhor',
 'ability',
 'able',
 'abound',
 'about',
 'above',
 'absolutel',
 'absolutely',
 'ac',
 'accept',
 'acceptable',
 'access',
 'accessable',
 'accessing',
 'accessory',
 'accessoryone',
 'accidentally',
 'accompanied',
 'according',
 'activate',
 'activated',
 'activesync',
 'actually',
 'ad',
 'adapter',
 'adapters',
 'add',
 'addition',
 'additional',
 'address',
 'adhesive',
 'adorable',
 'advertised',
 'advise',
 'after',
 'again',
 'against',
 'aggravating',
 'ago',
 'alarm',
 'all',
 'allot',
 'allow',
 'allowing',
 'allows',
 'almost',
 'alone',
 'along',
 'alot',
 'also',
 'although',
 'aluminum',
 'always',
 'am',
 'amazed',
 'amazing',
 'amazon',
 'amp',
 'ample',
 'an',
 'and',
 'angeles',
 'angle',
 'another',
 'answer',
 'ant

Посмотрим текст первого сообщения и какие признаки ему соответствуют.

In [700]:
print(ds.iloc[0].text)

So there is no way for me to plug it in here in the US unless I go by a converter.


In [701]:
X[0]

<1x1847 sparse matrix of type '<class 'numpy.int64'>'
	with 18 stored elements in Compressed Sparse Row format>

In [702]:
type(X)

scipy.sparse.csr.csr_matrix

In [703]:
X[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [704]:
X[X[0].nonzero()]

matrix([[1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int64)

In [705]:
X[0].nonzero()[1]

array([ 367,  233,  711, 1702, 1714, 1604,  762,  814,  857, 1212, 1640,
        993,  653, 1766, 1074,  854, 1609, 1491], dtype=int32)

In [706]:
feature_names = np.array(vectorizer.get_feature_names())
feature_names[X[0].nonzero()[1]]

array(['converter', 'by', 'go', 'unless', 'us', 'the', 'here', 'in', 'it',
       'plug', 'to', 'me', 'for', 'way', 'no', 'is', 'there', 'so'], 
      dtype='<U15')

### Построим линейный классификатор и посчитаем точность определения тональности

In [707]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

for c in [0.01, 0.1, 0.5, 1, 5, 10, 100, 200, 500, 1000, 10000, 15000, 20000, 100000]:
    cls = LogisticRegression(C=c)
    res = cross_val_score(cls, X, ds.is_positive, scoring="accuracy", cv=5)
    print(c, '\t', np.mean(res), np.std(res))

0.01 	 0.734 0.0390384425919
0.1 	 0.786 0.0193390796058
0.5 	 0.804 0.0168522995464
1 	 0.808 0.0186010752377
5 	 0.825 0.0197484176581
10 	 0.825 0.0144913767462
100 	 0.819 0.0135646599663
200 	 0.822 0.0128840987267
500 	 0.823 0.0120830459736
1000 	 0.819 0.0165529453572
10000 	 0.816 0.012409673646
15000 	 0.822 0.013638181697
20000 	 0.818 0.00979795897113
100000 	 0.812 0.0172046505341


In [693]:
res

array([ 0.84,  0.81,  0.79,  0.8 ,  0.82])

### Посмотрим, какие признаки оказывают наибольшее влияние при классификации

In [645]:
# Обучим классификатор на подвыборке
fraction = 900
cls.fit(X[:fraction], ds.is_positive[:fraction])

coef = cls.coef_[0]
order = np.argsort(abs(coef))[::-1]

for o in order[:60]:
    print(coef[o], '\t', vectorizer.get_feature_names()[o])

-13.8840576642 	 not
13.7612563736 	 best
-12.8207107234 	 starter
12.6904471954 	 definitely
-12.374473582 	 disappointed
12.02020772 	 love
-11.9827284451 	 poor
-11.2904635879 	 return
-11.1773888945 	 beware
10.8302019676 	 easy
-10.6422382089 	 disappointing
10.475943933 	 excellent
-10.4736754908 	 doesn
10.3660775432 	 great
-10.2335392006 	 unreliable
-10.0920104231 	 displeased
-9.9648946031 	 months
-9.82721479304 	 buying
9.77869536813 	 pleased
-9.73468534636 	 horrible
9.64971216661 	 worthwhile
-9.62329488683 	 difficult
9.59481999625 	 incredible
9.45302272312 	 nice
-9.45253724531 	 bad
9.37783430485 	 good
9.28583661453 	 infatuated
-9.25244570682 	 none
-9.01986958341 	 terrible
8.81921000326 	 sturdy
8.78895370215 	 works
-8.64117277341 	 worthless
-8.60210078713 	 infuriating
-8.58185449948 	 aggravating
-8.57170406361 	 disappointment
-8.48609880327 	 dirty
8.44762807176 	 than
-8.42840209947 	 headphones
-8.39087580028 	 wired
-8.35647590039 	 first
8.33616575838 

In [646]:
# Признаки, которые наш классификатор посчитал бесполезными
for o in order[-20:]:
    print(coef[o], '\t', vectorizer.get_feature_names()[o])

0.0 	 show
0.0 	 blew
0.0 	 allot
0.0 	 leaf
0.0 	 lense
0.0 	 leopard
0.0 	 satisifed
0.0 	 z500a
0.0 	 loops
0.0 	 alot
0.0 	 seen
0.0 	 loose
0.0 	 securely
0.0 	 looses
0.0 	 seat
0.0 	 lose
0.0 	 saved
0.0 	 bend
0.0 	 loudest
0.0 	 lightly


### Посмотрим на примеры работы классификатора 

In [649]:
sample = vectorizer.transform(["good razor", "bad razor", "razor", "good"])
cls.predict(sample)

array([1, 0, 0, 1])

In [650]:
cls.predict_proba(sample)

array([[  1.09651551e-04,   9.99890348e-01],
       [  9.99939468e-01,   6.05319758e-05],
       [  5.64572630e-01,   4.35427370e-01],
       [  1.79667155e-04,   9.99820333e-01]])

In [651]:
ds['prediction'] = cls.predict(X)
ds['prediction_proba'] = cls.predict_proba(X)[:,1]
ds['confidence'] = np.abs(ds.prediction_proba - 0.5)*2

In [652]:
ds[fraction:].head()

,text,is_positive,prediction,prediction_proba,confidence
900,"This was utterly confusing at first, which caused me to lose a couple of very, very important contacts.",0,0,0.000002,0.999996
901,Terrible phone holder.,0,0,0.000227,0.999545
902,The cutouts and buttons are placed perfectly.,1,1,0.999971,0.999943
903,I love being able to use one headset for both by land-line and cell.,1,1,1.000000,1.000000
904,Problem is that the ear loops are made of weak material and break easily.,0,0,0.000883,0.998234


In [653]:
ds.sort_values('confidence', ascending=False).tail()

,text,is_positive,prediction,prediction_proba,confidence
930,Never got it!!!!!,0,1,0.617510,0.235021
978,It fits so securely that the ear hook does not even need to be used and the sound is better directed through your ear canal.,1,0,0.409162,0.181676
977,":-)Oh, the charger seems to work fine.",1,0,0.413917,0.172167
973,Lousy product.,0,0,0.430544,0.138912
917,Leopard Print is wonderfully wild!.,1,1,0.523009,0.046018


### Посмотрим на ошибки классификатора

In [654]:
errors = ds.loc[ds.is_positive != ds.prediction]
errors.sort_values('confidence', ascending=False)

,text,is_positive,prediction,prediction_proba,confidence
934,"You get extra minutes so that you can carry out the call and not get cut off.""",1,0,1.383152e-14,1.000000
914,"My phone sounded OK ( not great - OK), but my wife's phone was almost totally unintelligible, she couldn't understand a word being said on it.",0,1,1.000000e+00,1.000000
911,So I bought about 10 of these and saved alot of money.,1,0,4.948956e-06,0.999990
946,I have had mine for about a year and this Christmas I bought some for the rest of the family.,1,0,5.010142e-06,0.999990
908,"I can hear while I'm driving in the car, and usually don't even have to put it on it's loudest setting.",1,0,7.868972e-06,0.999984
985,"There was so much hype over this phone that I assumed it was the best, my mistake.",0,1,9.999815e-01,0.999963
974,This phone tries very hard to do everything but fails at it's very ability to be a phone.,0,1,9.999417e-01,0.999883
962,Also makes it easier to hold on to.,1,0,3.101732e-03,0.993797
913,Couldn't figure it out,0,1,9.946530e-01,0.989306
992,Lasted one day and then blew up.,0,1,9.884404e-01,0.976881


In [648]:
# Посмотрим, почему плохо классифицируется пример
sample.nonzero()[1]
sample = vectorizer.transform(["It also had a new problem"])

print(cls.intercept_)
for f, c in zip( [vectorizer.get_feature_names()[i] for i in sample.nonzero()[1]], cls.coef_[0][sample.nonzero()[1]] ):
    print ( f, c )

also 0.677365228156
had 1.8161586742
it 0.385126593732
new 3.73079187999
problem -1.84626019089


### N-граммы

Раскомментируйте настройку CountVectorizer выше и прогоните ноутбук еще раз:

Среди важных признаков появляются двусловные. Качество возрастает, но незначительно.<br/>
Это может быть связано с тем, что количество признаков резко возрастает и у классификатора появляется больше возможностей переобучиться.

### Неустойчивость при смене корпуса

Теперь применим наш классификатор к другому корпусу и посмотрим получившееся на качество

In [655]:
# Вспомним, какие еще у нас есть файлы
!ls 'sentiment labelled sentences/'

amazon_cells_labelled.txt readme.txt
imdb_labelled.txt         yelp_labelled.txt


In [656]:
imdb = pd.read_csv("sentiment labelled sentences/imdb_labelled.txt", 
                   delimiter="\t", names=("text", "is_positive"))
yelp = pd.read_csv("sentiment labelled sentences/yelp_labelled.txt", 
                   delimiter="\t", names=("text", "is_positive"))

In [657]:
X_imdb = vectorizer.transform(imdb.text)
X_yelp = vectorizer.transform(yelp.text)
imdb['prediction'] = cls.predict(X_imdb)
yelp['prediction'] = cls.predict(X_yelp)

#imdb.loc[imdb.is_positive != imdb.prediction]

In [658]:
from sklearn.metrics import accuracy_score
print(accuracy_score(imdb.is_positive, imdb.prediction))
print(accuracy_score(yelp.is_positive, yelp.prediction) )

0.636363636364
0.724


Качество сильно проседает при смене корпуса. Это частая проблема при работе с текстами, о которой нужно помнить.

### TF*IDF

In [660]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(ds.text)

for c in [0.01, 0.1, 0.5, 1, 5, 10, 100, 200, 500, 1000, 10000, 15000, 20000, 100000]:
    cls = LogisticRegression(C=c)
    res = cross_val_score(cls, X_tfidf, ds.is_positive, scoring="accuracy", cv=5)
    print(np.mean(res), np.std(res))

0.789 0.0346987031458
0.789 0.0351283361405
0.803 0.0254165300543
0.813 0.0227156333832
0.827 0.0208806130178
0.83 0.0154919333848
0.824 0.0128062484749
0.829 0.010677078252
0.828 0.0132664991614
0.829 0.012409673646
0.829 0.012
0.829 0.012
0.828 0.0128840987267
0.823 0.0201494416796


В случае с короткими текстами TF*IDF вряд ли сильно поможет, т.к. tf == 1 почти всегда.<br/>
Получается, помочь может только idf, который увеличивает или уменьшает вес признака для всех сэмплов сразу.

### Pipeline

In [661]:
from sklearn.pipeline import Pipeline

text_processing_pipeline = Pipeline([
        ('Vectorizer', TfidfVectorizer(ngram_range=(1, 2))),
        ('Classifier', LogisticRegression(C=500))
    ])

res = cross_val_score(text_processing_pipeline, ds.text, ds.is_positive, scoring="accuracy", cv=5)
print(np.mean(res), np.std(res))

0.833 0.0273130005675


### Бейзлайн по тональности

In [ ]:
# Объединим все три датасета в один

In [664]:
reviews = ds.append(imdb).append(yelp)

In [665]:
reviews.shape

(2748, 5)

In [666]:
res = cross_val_score(text_processing_pipeline, reviews.text, reviews.is_positive, scoring="accuracy", cv=5)
print(np.mean(res), np.std(res))

0.826031929782 0.0270132991861


### Замечание о качестве датасета

### См. бонус

## Классификация по темам

In [675]:
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'), categories=['comp.windows.x', 'rec.autos', 'sci.med'])

In [676]:
newsgroups.target

array([2, 0, 0, ..., 2, 2, 1])

In [677]:
# Имена классов
newsgroups.target_names

['comp.windows.x', 'rec.autos', 'sci.med']

In [678]:
vectorizer = CountVectorizer()#ngram_range=(2,2))
X = vectorizer.fit_transform(newsgroups.data)

In [679]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

#for c in [0.01, 0.1, 0.5, 1, 5, 10, 100, 200, 500, 1000, 10000, 15000, 20000, 100000]:
cls = LogisticRegression()#C=c)
res = cross_val_score(cls, X, newsgroups.target, scoring="f1_macro", cv=3)
print(np.mean(res), np.std(res))

0.87411311916 0.00541315872889


## Самостоятельная работа

Вам предстоит улучшить качество классификаторов относительно полученных выше бейзлайнов.<br/>
Ниже предложены некоторые идеи, которые могут помочь вам добиться этого.<br/>
Воспользуйтесь ими, или предложите свой подход<br/>


По результатам напишите краткий отчет об экспериментах и результатах на andy.belov@gmail.com с пометкой [MIPT_NLP_2017] в теме

### Настройка гиперпараметров классификатора

Подберите наилучшие параметры векторайзера (напр. использовать tf*idf, n-граммы) и классификатора (напр. регуляризатор и коэффициент регуляризации), воспользовавшись модулем grid_search библиотеки scikit-learn<br/>
http://scikit-learn.org/stable/modules/grid_search.html <br/>
http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

### Подбор классификаторов

Попробуйте использовать другие классификаторы из библиотеки scikit-learn (не забывайте для них тоже подбирать наилучшие гиперпараметры)<br/>
http://scikit-learn.org/stable/supervised_learning.html

### Фильтрация стоп-слов

Исключите из признаков стоп-слова (см. семинар 1). Это может уменьшить переобучение классификатора.

### Использование стемминга и лемматизации

Предобработайте текст, использовав стемминг или лемматизацию (см. семинар 1).


### Отбор признаков

http://scikit-learn.org/stable/modules/feature_selection.html

### Предобработка

Настройте токенизатор (можно использовать nltk см. семинар 1), очистите тексты от мусора, чисел (попробуйте заменить их на 00) и т.п.

## Бонус: классификация твитов

Вам понадобится установить библиотеку tweepy и зарегистрировать приложение на сайте twitter (это бесплатно)

In [681]:
# Простой пример кода, позволяющего стримить твиттер

from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json

# Чтобы пользоваться api twitter, зарегистрируйтесь и создайте новое приложение на сайте twitter
# и впишите полученные consumer key, consumer secret, access token, access secret сюда.
#ckey = ""
#csecret = ""
#atoken = ""
#asecret = ""

# Здесь я импортирую свои credentials из своего файла. Вам эта строчка не нужна.
#from twitter_settings import ckey, csecret, atoken, asecret

# Сюда будем складывать твиты.
tweets = []

class listener(StreamListener):
    def on_data(self, data):
        all_data = json.loads(data)
        tweet = all_data["text"]
        tweets.append(tweet)
        print(tweet)
        print("="*100)
        return(True if len(tweets) < 20 else False)

    def on_error(self, status):
        print(status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["iphone"])

1 Pcs Lightning Charger Cable Saver Protector Accessory For Apple iPhone 5 5S 6S https://t.co/pR7SReDJFV https://t.co/cM2l4bG9Vh
Fashion Leather Stand Wallet Card Wrist Strap Filp Cover Case For iPhone 5S/5G https://t.co/mde9O1VCKI https://t.co/yFBZvYaSqg
HURLEY GOLD LOGO SKATE LAPTOP CAR STICKER DECAL VINYL iphone #67YearVinyl https://t.co/9ZizFa2EhC
New Beer Pattern Slim Soft TPU Fashion Phone Case Cover Skin for Iphone 5 5S https://t.co/y2EnZa8e1F https://t.co/uKLmZRsTEE
The Office Insider program for iPhone and iPad kicked off at the end of January in which users can test Word,... https://t.co/yjaasLjKUv
RT @___rkp_12: モイ！iPhoneからキャス配信中 -カラオケ♡ https://t.co/2ihHDY3fM7
おにゅーiPhoneケース🍟 https://t.co/5zybqyPHcC
For Apple iPhone 5 5G 5S Ultra Thin Transparent Clear Protect Case Cover Skin CA https://t.co/KlHW2HCPQb https://t.co/7EvH5c8zwF
2pc Micro USB to Lightning 8 Pin Charger Converter AdaG2er Fr iPhone 5s/6s G2 https://t.co/a5OJiKbXbb https://t.co/wfObq72uqz
Fashion Leather Stand Wall

In [710]:
cls

LogisticRegression(C=100000, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [709]:
# Воспользуемся обученным ранее классификатором для оценки тональности твита.
probs = cls.predict_proba(vectorizer.transform(tweets))[:,1]
for t, p in zip(tweets, probs):
    print(p, '\t', t, '\n')

NotFittedError: Call fit before prediction

## Классификация сообщений вконтакте

В качестве эксперимента можете обучить классификатор на русском датасете и применить его к своим сообщениям из ВКонтакте: попробуйте найти самые негативные и позитивные из них по мнению классификатора<br/>
Датасет: http://study.mokoron.com<br/>
API: https://vk.com/dev/messages.getHistory<br/>

## Другие интересные датасеты

Тональность отзывов на кинофильмы. <br/>
Есть отзывы на несколько предложений, есть разметка отдельных предложений, есть разметка предложений на эмоционально окрашенные и не окрашенные. Это может помочь с твиттером.<br/>
https://www.cs.cornell.edu/people/pabo/movie-review-data/

Классификация вопросов по типу<br/>
http://cogcomp.cs.illinois.edu/Data/QA/QC/

Reuters - классификация статей<br/>
https://archive.ics.uci.edu/ml/datasets/Reuters-21578+Text+Categorization+Collection